In [1034]:
import pandas as pd
import numpy as np
from datetime import datetime
from torch.utils.data import DataLoader, Dataset
import ast
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch.nn as nn
import keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

# Data Pre-Processing

In [1035]:
# Load Datasets

# Training datasets
cgm_train = pd.read_csv('cgm_train.csv')
image_train = pd.read_csv('img_train.csv')
demo_viome_train = pd.read_csv('demo_viome_train.csv')
label_train = pd.read_csv('label_train.csv')

# Test datasets
cgm_test = pd.read_csv('cgm_test.csv')
image_test = pd.read_csv('img_test.csv')
demo_viome_test = pd.read_csv('demo_viome_test.csv')
label_test = pd.read_csv('label_test_breakfast_only.csv')

### Pre-Process CGM Data (Time-Series Glucose Levels)

In [ ]:
# Updating missing values with mean

# Function to check if CGM Data is an empty array
def is_cgm_data_empty(row):
    try:
        cgm_list = ast.literal_eval(row['CGM Data'])
        return len(cgm_list) == 0
    except:
        return True

# Function to filter out rows where CGM Data is empty
cgm_train = cgm_train[~cgm_train.apply(is_cgm_data_empty, axis=1)]

# Convert Breakfast and Lunch times to datetime
# Convert Breakfast and Lunch times to datetime
cgm_train['Breakfast Time'] = pd.to_datetime(cgm_train['Breakfast Time'], errors='coerce')
cgm_train['Lunch Time'] = pd.to_datetime(cgm_train['Lunch Time'], errors='coerce')

# Extract the date part and time part separately
cgm_train['Breakfast Date'] = cgm_train['Breakfast Time'].dt.date
cgm_train['Lunch Date'] = cgm_train['Lunch Time'].dt.date

# Function to calculate mean time while preserving date
def mean_time(times):
    # Convert the times to minutes since midnight
    total_seconds = sum([t.hour * 3600 + t.minute * 60 + t.second for t in times if pd.notna(t)])
    mean_seconds = total_seconds // len([t for t in times if pd.notna(t)])
    
    # Convert the mean seconds back to HH:MM:SS
    return pd.to_datetime(mean_seconds, unit='s').time()

# Calculate the mean breakfast and lunch times per Subject ID
mean_times = cgm_train.groupby('Subject ID')[['Breakfast Time', 'Lunch Time']].apply(
    lambda group: pd.Series({
        'Breakfast Time': mean_time(group['Breakfast Time']),
        'Lunch Time': mean_time(group['Lunch Time'])
    })
)

# Merge the date back with the mean times
cgm_train['Breakfast Time'] = cgm_train['Breakfast Date'].astype(str) + ' ' + cgm_train['Breakfast Time'].fillna(cgm_train['Subject ID'].map(mean_times['Breakfast Time'])).astype(str)
cgm_train['Lunch Time'] = cgm_train['Lunch Date'].astype(str) + ' ' + cgm_train['Lunch Time'].fillna(cgm_train['Subject ID'].map(mean_times['Lunch Time'])).astype(str)

# Convert back to datetime
cgm_train['Breakfast Time'] = pd.to_datetime(cgm_train['Breakfast Time'])
cgm_train['Lunch Time'] = pd.to_datetime(cgm_train['Lunch Time'])

# Drop the helper date columns
cgm_train.drop(columns=['Breakfast Date', 'Lunch Date'], inplace=True)


cgm_train.to_csv('cleaned_data.csv')

In [1037]:
# # Function to check if CGM Data is an empty array
# def is_cgm_data_empty(row):
#     try:
#         cgm_list = ast.literal_eval(row['CGM Data'])
#         return len(cgm_list) == 0
#     except:
#         return True

# # Function to filter out rows where CGM Data is empty
# cgm_train = cgm_train[~cgm_train.apply(is_cgm_data_empty, axis=1)]

# # Handle missing breakfast and lunch times
# cgm_train['Breakfast Time'] = pd.to_datetime(cgm_train['Breakfast Time'], errors='coerce')
# cgm_train['Lunch Time'] = pd.to_datetime(cgm_train['Lunch Time'], errors='coerce')

# # Extract CGM data as list of tuples, convert to list of time series values
# cgm_train['CGM Data'] = cgm_train['CGM Data'].apply(lambda x: eval(x) if isinstance(x, str) else [])

# # Extract features from CGM data (flatten the time and glucose values)
# def extract_cgm_features(cgm_data):
#     times = [entry[0] for entry in cgm_data]
#     glucose_levels = [entry[1] for entry in cgm_data]
#     return times, glucose_levels

# cgm_train['CGM Times'], cgm_train['CGM Levels'] = zip(*cgm_train['CGM Data'].apply(extract_cgm_features))

# # Normalize glucose levels
# scaler = StandardScaler()
# cgm_train['CGM Levels'] = cgm_train['CGM Levels'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)).flatten())

Rows with empty CGM data have been removed. TODO: Put in function to reuse for test

In [1038]:
# # We need to pad the sequences to a fixed length for GRU input
# max_sequence_length = 300  # Define a maximum length for the sequences
# cgm_train['Padded CGM Levels'] = pad_sequences(cgm_train['CGM Levels'], maxlen=max_sequence_length, padding='post', value=0, dtype='float32').tolist()

# # Mask labels: We will use NaN or a predefined mask value for missing times -- 1
# cgm_train['Breakfast Time Masked'] = cgm_train['Breakfast Time'].isna().astype(int)
# cgm_train['Lunch Time Masked'] = cgm_train['Lunch Time'].isna().astype(int)

# # Prepare the target variable: encode the time values for breakfast and lunch
# def encode_times(time_column):
#     return (time_column - pd.Timestamp('2021-09-18')) // pd.Timedelta('1s')

# # Filter rows where both Breakfast and Lunch times are missing (i.e., both masks are 0)
# filtered_cgm_train = cgm_train[(cgm_train['Breakfast Time Masked'] == 0) & (cgm_train['Lunch Time Masked'] == 0)].copy()

# # Encode breakfast and lunch times only for rows where both are missing
# filtered_cgm_train['Breakfast Time Encoded'] = encode_times(filtered_cgm_train['Breakfast Time'])
# filtered_cgm_train['Lunch Time Encoded'] = encode_times(filtered_cgm_train['Lunch Time'])

# time_scaler = MinMaxScaler()

# # Reshape and scale both 'Breakfast Time Encoded' and 'Lunch Time Encoded'
# filtered_cgm_train[['Breakfast Time Encoded', 'Lunch Time Encoded']] = time_scaler.fit_transform(
#     filtered_cgm_train[['Breakfast Time Encoded', 'Lunch Time Encoded']]
# )

# X_train = np.array(filtered_cgm_train['Padded CGM Levels'].tolist())
# y_train = filtered_cgm_train[['Breakfast Time Encoded', 'Lunch Time Encoded']].values

In [1039]:
# # Reshape X_train to (samples, time steps, features)
# X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # (samples, time steps, features)

# # Define the GRU model
# def create_gru_model(input_shape):
#     model = Sequential([
#         Input(shape=input_shape),
#         GRU(32, activation='relu'),
#         Dense(2)  # Output two values: breakfast and lunch times
#     ])
#     model.compile(optimizer=Adam(learning_rate=0.001, clipvalue=1.0), loss='mse')
#     return model

# # Create and compile the model
# model = create_gru_model((X_train_reshaped.shape[1], 1))

# # Train the model
# model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


In [1040]:
# def predict_meal_times(model, X):
#     X = np.array(X.tolist()) if isinstance(X, pd.Series) else np.array(X)
#     X_reshaped = X.reshape((X.shape[0], X.shape[1], 1))
#     predictions = model.predict(X_reshaped)
#     reference_date = pd.Timestamp('2021-09-18')

#     # Decode predictions back to original scale using inverse transformation
#     decoded_times = time_scaler.inverse_transform(predictions)

#     # Add the decoded seconds back to the reference date
#     decoded_breakfast_timestamps = reference_date + pd.to_timedelta(decoded_times[:, 0], unit='s')
#     decoded_lunch_timestamps = reference_date + pd.to_timedelta(decoded_times[:, 1], unit='s')

#     decoded_predictions = pd.DataFrame({
#         'Predicted Breakfast Time': decoded_breakfast_timestamps,
#         'Predicted Lunch Time': decoded_lunch_timestamps
#     })

#     return decoded_predictions

# # Extract rows with missing breakfast or lunch times
# missing_data = cgm_train[cgm_train['Breakfast Time'].isna() | cgm_train['Lunch Time'].isna()]

# # Ensure that `Padded CGM Levels` is included in `missing_data`
# predict_missing = missing_data['Padded CGM Levels']
# missing_data_copy = missing_data.copy()

# # Make predictions for missing breakfast and lunch times
# predicted_times = predict_meal_times(model, predict_missing)

# # Reset indices for both DataFrames to align by row order
# missing_data_copy = missing_data_copy.reset_index(drop=True)
# predicted_times = predicted_times.reset_index(drop=True)

# # Add the 'Predicted Breakfast Time' column
# missing_data_copy['Predicted Breakfast Time'] = predicted_times['Predicted Breakfast Time']
# missing_data_copy['Predicted Lunch Time'] = predicted_times['Predicted Lunch Time']

In [1041]:
# np.savetxt('output.txt', y_breakfast_masked, delimiter=',')

In [1042]:
# class GRUPredictor(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers):
#         super(GRUPredictor, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size, 1)
    
#     def forward(self, x):
#         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
#         out, _ = self.gru(x, h0)
#         out = self.fc(out[:, -1, :])
#         return out
    
# model = GRUPredictor(input_size=1, hidden_size=64, num_layers=2)
# criterion = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# # Convert data to PyTorch tensors
# X_tensor = torch.FloatTensor(X.reshape(-1, max_sequence_length, 1))
# y_tensor = torch.FloatTensor(y_breakfast_masked_train)

# # Train the model
# num_epochs = 10
# for epoch in range(num_epochs):
#     outputs = model(X_tensor)
#     loss = criterion(outputs, y_tensor)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     print(f'Epoch: {epoch} Loss: {loss.item()}')

In [1043]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import GRU, Dense, Masking

# # Build the GRU model
# model = Sequential()

# # Masking layer to ignore padding during training
# model.add(Masking(mask_value=0., input_shape=(max_sequence_length, 1)))

# # GRU layers
# model.add(GRU(128, return_sequences=False))
# model.add(Dense(64, activation='relu'))

# # Output layer for predicting breakfast and lunch times (regression problem)
# model.add(Dense(1))

# # Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error')

# # Model summary
# model.summary()
 

In [1044]:
# # Prepare the data for training
# X = np.expand_dims(X, axis=-1)  # Add a channel dimension for GRU input

# # Split into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y_breakfast_masked, test_size=0.2, random_state=42)

# # Train the model
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# # Evaluate the model
# loss = model.evaluate(X_val, y_val)
# print(f'Validation Loss: {loss}')


In [1045]:
# # Predict missing breakfast times (masked values)
# predicted_breakfast_times = model.predict(X_val)

# predicted_breakfast_times

# Convert the predicted time in seconds back to datetime format
# predicted_breakfast_times = pd.to_datetime(predicted_breakfast_times, unit='s', origin='1970-01-01')

# # You can use a similar approach for lunch time prediction
# predicted_lunch_times = model.predict(X_val)
# predicted_lunch_times = pd.to_datetime(predicted_lunch_times, unit='s', origin='1970-01-01')


In [1046]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
# from sklearn.impute import SimpleImputer

# # Load dataset with proper delimiter (ensure '\t' for tab-separated values)
# file_path = 'demo_viome_train.csv'
# data = pd.read_csv(file_path, delimiter='\t')

# # Recheck column parsing
# if len(data.columns) == 1:
#     # If all data is in a single column, try splitting with a comma
#     data = pd.read_csv(file_path, delimiter=',')

# # Verify column names
# print("Columns in dataset after re-parsing:", data.columns)

# # Split the `Viome` column into individual features
# viome_split = data['Viome'].str.split(',', expand=True).astype(float)
# viome_split.columns = [f"Viome_{i}" for i in range(viome_split.shape[1])]

# # Drop the original Viome column and merge new features
# data = pd.concat([data.drop(columns=['Viome']), viome_split], axis=1)

# # Impute missing values for numeric columns
# numeric_cols = data.select_dtypes(include=[np.number]).columns
# imputer = SimpleImputer(strategy='mean')
# data[numeric_cols] = imputer.fit_transform(data[numeric_cols])

# # Normalize numeric data
# scaler = MinMaxScaler()
# data[numeric_cols] = scaler.fit_transform(data[numeric_cols])


# # Encode categorical columns
# categorical_cols = ['Gender', 'Race', 'Diabetes Status']
# encoder = OneHotEncoder(sparse_output=False, drop='first')  # Use sparse_output instead of sparse
# encoded_cats = pd.DataFrame(
#     encoder.fit_transform(data[categorical_cols]),
#     columns=encoder.get_feature_names_out(categorical_cols)
# )

# # Drop original categorical columns and merge encoded ones
# data = pd.concat([data.drop(columns=categorical_cols), encoded_cats], axis=1)

# # Final processed data
# print("Processed Data Shape:", data.shape)
# print("Processed Data Preview:")
# print(data.head())


In [1047]:
# import pandas as pd
# import numpy as np
# from PIL import Image

# # Load the dataset
# data = pd.read_csv("img_train.csv")  # Adjust the file path as necessary

# # Placeholder for missing images (a blank black image)
# def create_placeholder_image(size=(64, 64, 3)):
#     return np.zeros(size, dtype=np.float32)  # Normalized [0, 1] range

# # Function to preprocess image data
# def preprocess_image(img_data, size=(64, 64)):
#     try:
#         img_array = np.array(img_data, dtype=np.uint8)  # Ensure valid data type

#         # Check for empty image
#         if img_array.size == 0 or img_array.ndim != 3 or img_array.shape[2] != 3:
#             raise ValueError(f"Invalid or empty image dimensions: {img_array.shape}")

#         img_resized = np.array(Image.fromarray(img_array).resize(size))  # Resize
#         img_normalized = img_resized / 255.0  # Normalize pixel values to [0, 1]
#         return img_normalized
#     except Exception as e:
#         print(f"Error preprocessing image: {e}")
#         return create_placeholder_image(size)

# # Preprocess the dataset
# def preprocess_dataset(data):
#     # Define placeholder image
#     placeholder_image = create_placeholder_image()

#     # Create missingness indicators
#     data['Breakfast_Missing'] = data['Image Before Breakfast'].isnull().astype(int)
#     data['Lunch_Missing'] = data['Image Before Lunch'].isnull().astype(int)

#     # Iterate over rows to preprocess images
#     breakfast_images = []
#     lunch_images = []

#     for index, row in data.iterrows():
#         # Handle missing breakfast images
#         if pd.isnull(row['Image Before Breakfast']) or row['Image Before Breakfast'] == '[]':  # Check for empty list or NaN
#             breakfast_images.append(placeholder_image)
#         else:
#             try:
#                 img_data = eval(row['Image Before Breakfast'])  # Convert string to list
#                 breakfast_images.append(preprocess_image(img_data))
#             except Exception as e:
#                 print(f"Error at index {index}, breakfast: {e}")
#                 breakfast_images.append(placeholder_image)

#         # Handle missing lunch images
#         if pd.isnull(row['Image Before Lunch']) or row['Image Before Lunch'] == '[]':  # Check for empty list or NaN
#             lunch_images.append(placeholder_image)
#         else:
#             try:
#                 img_data = eval(row['Image Before Lunch'])  # Convert string to list
#                 lunch_images.append(preprocess_image(img_data))
#             except Exception as e:
#                 print(f"Error at index {index}, lunch: {e}")
#                 lunch_images.append(placeholder_image)

#     # Add preprocessed images back to the dataset
#     data['Processed_Breakfast_Images'] = breakfast_images
#     data['Processed_Lunch_Images'] = lunch_images

#     return data

# # Apply preprocessing
# processed_data = preprocess_dataset(data)

# # Save the processed dataset if needed
# # processed_data.to_pickle("processed_img_train.pkl")  # Save in pickle format for further use


In [1048]:
# import pandas as pd

# # Load the dataset
# label_data = pd.read_csv("label_train.csv")  # Adjust the file path as necessary

# # Step 1: Extract Output Labels
# output_labels = label_data[["Breakfast Calories", "Lunch Calories"]]

# # Step 2: Handle Missing Values in Labels
# # Replace missing values (if any) with the median
# output_labels = output_labels.fillna(output_labels.median())

# print(output_labels)

# # Step 3: Save the Extracted Labels
# # output_labels.to_csv("output_labels.csv", index=False)

# print("Output Labels Extracted and Saved!")
